# 第6回講義 宿題

## 課題
RNNを用いてIMDbのsentiment analysisを実装してみましょう。

ネットワークの形などは特に制限を設けませんし、今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません。

## 目標値
F値：0.85

## ルール
- 以下のセルで指定されている`train_data`以外の学習データは使わないでください。
- `test_data`の正解ラベルを参照しないでください。

## 提出方法
- 2つのファイルを提出していただきます。
  1. テストデータ (x_test) に対する予測ラベルを`submission_pred.csv`として保存し、Homeworkタブから**chap08**を選択して提出してください。
  2. それに対応するpythonのコードを`submission_code.py`として保存し、Homeworkタブから**chap08 (code)**を選択して提出してください。
    - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください。
    - writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください


- コードの内容を変更した場合は、1と2の両方を提出し直してください。

- なお、採点は1で行い、2はコードの確認用として利用します。(成績優秀者はコード内容を公開させていただくかもしれません。)

- **宿題の締め切りは【出題週の翌週水曜日24時】です。**

## 評価方法

- 予測ラベルの（`t_test`に対する）F値で評価します。
- 定時に評価しLeader Boardを更新します。
- 締切後のF値でLeader Boardを更新します。これを最終的な評価とします。

## データの読み込み（このセルは修正しないでください）

In [ ]:
import numpy as np
import torch
import torchtext.legacy
from torchtext.legacy import data
from torchtext.legacy import datasets
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import string
import re

rng = np.random.RandomState(1234)
random_state = 42


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 文章とラベルの両方に用意します
max_length = 256
TEXT = data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                  use_vocab=True, lower=True, include_lengths=True,
                  batch_first=True, fix_length=max_length,
                  init_token="<cls>", eos_token="<eos>")
LABEL = data.Field(sequential=False, use_vocab=True)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語


# データセットの作成

train_data, test_data = datasets.IMDB.splits(text_field=TEXT,
                                             label_field=LABEL)

for i in range(len(test_data)):
    if i % 2 == 0:
        test_data[i].label = "pos"
    else:
        test_data[i].label = "neg"

In [ ]:
train_data, valid_data = train_data.split(0.8)

## 実装

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

word_num = 3000
TEXT.build_vocab(train_data, max_size=word_num)
LABEL.build_vocab(train_data)

batch_size = 100

train_dl = torchtext.legacy.data.Iterator(train_data, batch_size=batch_size,
                                   train=True, sort=True)

valid_dl = torchtext.legacy.data.Iterator(valid_data, batch_size=batch_size,
                                   train=False, sort=False)

test_dl = torchtext.legacy.data.Iterator(test_data, batch_size=batch_size,
                                  train=False, sort=False)


def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))


class Embedding(nn.Module):
    # WRITE ME

class SequenceTaggingNet(nn.Module):
    # WRITE ME


emb_dim = 100
hid_dim = 50
n_epochs = 10
device = 'cuda'

net = SequenceTaggingNet(word_num + 4, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters())

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    net.train()
    n_train = 0
    acc_train = 0
    for mini_batch in train_dl:

        # WRITE ME

        losses_train.append(loss.tolist())

        n_train += t.size()[0]

    # Valid
    t_valid = []
    y_pred = []
    net.eval()
    for mini_batch in valid_dl:

        # WRITE ME

        t_valid.extend(t.tolist())
        y_pred.extend(pred.tolist())

        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train Loss: {:.3f}, Valid Loss: {:.3f}, Validation F1: {:.3f}'.format(
        epoch,
        np.mean(losses_train),
        np.mean(losses_valid),
        f1_score(t_valid, y_pred, average='macro')
    ))


In [ ]:
net.eval()

y_pred = []
for mini_batch in test_dl:

    x = mini_batch.text[0].to(device)
    len_seq = mini_batch.text[1].to(device)
    h = net(x, torch.max(len_seq), len_seq)
    y = torch.sigmoid(h).squeeze()

    pred = y.round().squeeze()  # 0.5以上の値を持つ要素を正ラベルと予測する

    y_pred.extend(pred.tolist())


submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210520/submission_pred.csv', header=True, index_label='id')